# e-H scattering
[Link to tutorial](https://www.octopus-code.org/documentation/13/tutorial/model/e-h_scattering/)

In this tutorial, we will show how to model the problem of an electron wavepacket scattering on an hydrogen atom using Octopus. In order to speed up the calculations and make it easier to plot the different quantities of interest, we will do the calculation in 1D. It should be straightforward to change this example to simulate a “real” 3D hydrogen atom.

In [ ]:
import holoviews as hv
from holoviews import opts

from postopus import Run

hv.extension("bokeh", "matplotlib")

In [ ]:
!mkdir -p 4-e-H-scattering

In [ ]:
cd 4-e-H-scattering

## Ground state

The first thing to do is to tell Octopus what we want it to do. We will start with an input file containing only the description of a 1D hydrogen atom, modeled by a soft Coulomb potential:

In [ ]:
%%writefile inp


stdout = 'stdout_gs.txt'
stderr = 'stderr_gs.txt'

FromScratch = yes
CalculationMode = gs

Dimensions = 1

Radius = 50
Spacing = 0.15

%Species
 "H" | species_soft_coulomb | softening | 1 | valence | 1
%

%Coordinates
 "H" | -10
%

SpinComponents = polarized

ExtraStates = 1

%Occupations
 1 | 0
 0 | 0
%

These input variables should already be familiar as they have been explained in previous tutorials. Here we simply note the following:

* [Spacing](https://www.octopus-code.org/documentation//13/variables/mesh/spacing): We employed a large box. This is needed for the time-dependent simulation, and, as we will see, is in fact not large enough for correctly describing our e-H problem for even 1 fs.
* [ExtraStates](https://www.octopus-code.org/documentation//13/variables/states/extrastates): We requested one extra state. This is not really needed for the ground-state calculation, but is crucial for the time-dependent run bellow, as there we will replace this unoccupied state by the electron wavepacket.
* [Occupations](https://www.octopus-code.org/documentation//13/variables/states/occupations): We have fixed the occupations to be one in the first spin channel. This is needed for the TD run, as we will see, and also because otherwise the code might get to a different ground state than the one we are interested.

In [ ]:
!octopus

## Output

Now one can execute this file by running **Octopus**. Here are a few things worthy of a closer look in the standard output. We are doing an LDA calculation, as we want to investigate the role of electron-electron interaction here, and, as a possible application, benchmark the performance of exchange-correlation functionals. As we are doing a one dimensional calculation, we see that **Octopus** has selected the corresponding 1D version of the LDA functional:

In [ ]:
!cat stdout_gs.txt | grep -A 15 "[*] Theory Level [*]"

## e-H-scattering

We are now ready to perform a simulation of the e-H scattering problem. For this, we use the following input file:

In [ ]:
%%writefile inp

stdout = 'stdout_td.txt'
stderr = 'stderr_td.txt'
FromScratch = yes
CalculationMode = td

Dimensions = 1

Radius = 50
Spacing = 0.15

%Species
"H" | species_soft_coulomb | softening | 1 | valence | 1
%

%Coordinates
"H" | -10
%

SpinComponents = polarized

ExtraStates = 1

ExcessCharge = -1
%Occupations
 1 | 1
 0 | 0
%
RestartFixedOccupations = no

a = 0.1
x0 = 10.0
p = -1.5
%UserDefinedStates
  1 | 2 | 1 | formula | "exp(-a*(x-x0)^2+i*p*(x-x0))"
%

%Output
 density
 potential
%
OutputFormat = axis_x
TDPropagationTime = 0.8 * femtosecond

In order to introduce the electron wavepacket to scatter on the hydrogen atom, we have done the following change:
* [ExcessCharge](https://www.octopus-code.org/documentation//13/variables/states/excesscharge): We introduce an extra electron in our simulation by setting this variable to -1.
* [Occupations](https://www.octopus-code.org/documentation//13/variables/states/occupations): We force the extra electron to be on the second state of the first spin channel.
* [RestartFixedOccupations](https://www.octopus-code.org/documentation//13/variables/states/restartfixedoccupations)"  = no: Note, that we need to override the default behaviour to keep the occupations from the restart file.
* [UserDefinedStates](https://www.octopus-code.org/documentation//13/variables/states/userdefinedstates): We are replacing the originally unoccupied state obtained after the GS calculation by a user defined state, which is a Gaussian of the form $
\phi(x) = e^{-\alpha (x-x_0)^2+i p (x-x_0)}, $ where $\alpha$ is taken to be $\alpha=0.1$ here, $x_0$ is the initial position of the wavepacket, taken to be at $x_0=-10$ a.u., and $p$ its velocity, taken to be $p=-1.5$ a.u..


The result of the calculation is shown in Fig. 1. What we are interested in is the density of the up channel, where both electrons are, as well as the corresponding potential. These quantities are found in the file 'output_iter/td.XXXXXXX/density-sp1.y=0,z=0' and  'output_iter/td.XXXXXXX/vxc-sp1.y=0,z=0'.
We note a few interesting details. First, we observe no significant reflection of the wavepacket density on the hydrogen atom. This is a known deficiency of the adiabatic approximation.
Moreover, towards the end of the simulation, both the density and the exchange-correlation potential show strong oscillations on the left-hand side of the simulation.
This is due to the artificial reflection of the electron wavepacket at the border of the simulation box.
This can be easily fixed by double the size of the box ([Radius](https://www.octopus-code.org/documentation//13/variables/mesh/simulation_box/radius) = 100), and/or by using absorbing boundaries.

In [ ]:
!octopus

In [ ]:
!cat stdout_td.txt

In [ ]:
run = Run(".")

## Create plot

In [ ]:
nx = run.default.td.density_sp1()

In [ ]:
hv_nx = hv.Dataset(nx)

In [ ]:
vxc = run.default.td.vxc_sp1()

In [ ]:
hv_vxc = hv.Dataset(vxc)

In [ ]:
layout = (
    hv_nx.to(hv.Curve, "x", "density_sp1") + hv_vxc.to(hv.Curve, "x", "vxc_sp1")
).cols(1)
layout.opts(opts.Curve(width=450, height=450, framewise=True))
layout

[Go to *5-Kronig-Penney-Model.ipynb*](5-Kronig-Penney-Model.ipynb)